In [63]:
import numpy as np
import pickle
import random
import pandas as pd
corr = None
ifTest = False
Ztr = {}
Zte = {}


def returnNext(state, action):
    # Returns (priceChange, nextState) as a tuple
    #nextState is actually a 2-tuple representing the state
    a,b = state
    L = []
    if ifTest:
        L = Zte[(a,b)]
    else:
        L = Ztr[(a,b)]
    r = L[random.randint(0,len(L)-1)]
    pricc = None
    if action==-1:
        if r[0]==0:
            pricc =  r[4]
        elif r[0]==1:
            pricc =  r[3]
        else:
            pricc =  -r[3]
    elif action==1:
        if r[2]==0:
            pricc =  r[4]
        elif r[2]==1:
            pricc =  r[3]
        else:
            pricc =  -r[3]
    else:
        if r[1]==0:
            pricc =  r[4]
        elif r[1]==1:
            pricc =  r[3]
        else:
            pricc =  -r[3]
    num = 3*(10*a + b) + action + 1

    F = corr[num,:]
    ch = np.random.choice(np.arange(0,100),p = list(F))
    ca = ch // 10
    cb = ch % 10
    return (pricc,(ca,cb))

In [64]:
# total number of States and Actions
n_states =100
n_actions = 3

In [66]:
# initial Q-table
Q = np.zeros([n_states, n_actions])
epsilons = []

# learning rate
alpha = 1.0

In [67]:
# discount factor
gamma = 1.0

# e-greedy exploitation
epsilon = 0.9
epsilon_decay = 0.01
epsilon_final = 0.01

In [71]:
# training parameters
n_episodes = 200
n_steps = 100

In [70]:
def getState(state, epsilon):
    global Q
    p = np.random.rand()
            
    action =None
    if p>epsilon:
        rand_values = Q[state]+ np.random.rand(1,n_actions)/1000
        action = np.argmax(rand_values)-1

    else:
        action = np.random.randint(n_actions)-1
        
    return action

In [76]:
# Train your Markov Decision Process, make use of the returnNext function to model and act on the priceChanges dependence on the states and actions
def Train():
    
    global n_episodes, n_steps, epsilon, epsilon_decay, epsilon_final, gamma, alpha, epsilons, Q
    
    for i in range(n_episodes):
        state =0
        cum_reward =0
        done =False
        
        inventory = []
        price =100
        
        for j in range(n_steps):
            p = np.random.rand()
            
            if p>epsilon:
                rand_values = Q[state]+ np.random.rand(1,n_actions)/1000
                action = np.argmax(rand_values)-1
            
            else:
                action = np.random.randint(n_actions)-1
            
            
            
            if epsilon > epsilon_final:
                epsilon*=(1-epsilon_decay)
            
            picc, nxt_state = returnNext((state//10,state%10), action)
            
            new_state =nxt_state[0]*10+nxt_state[1]
                                         
            reward =0
            price+=picc
            if action==1:
                inventory.append(price)
            elif action ==-1 and len(inventory)>0:
                bought_price = inventory.pop(0)
                reward = price-bought_price
            
            Q[state][action] = ((1-alpha)*Q[state][action]) + alpha*(reward + gamma*np.max(Q[new_state]))
            
            cum_reward+=reward
            state = new_state
            
            
            print(state, action, price, reward, inventory)

        
ifTest  = False
Train()

79 -1 100.31883818394381 0 []
17 1 101.11881400909566 0 [101.11881400909566]
67 0 101.30360487567839 0 [101.11881400909566]
53 0 103.00384018407325 0 [101.11881400909566]
21 -1 103.77119931769853 2.6523853086028737 []
45 1 103.90690085265953 0 [103.90690085265953]
47 -1 103.15401692113353 -0.7528839315260001 []
65 1 103.04716817307377 0 [103.04716817307377]
56 -1 103.52162506302771 0.47445688995394164 []
2 0 104.4102108948494 0 []
42 -1 103.75865300078286 0 []
47 0 104.61818492895361 0 []
54 1 104.86922885641631 0 [104.86922885641631]
46 0 106.01656842288243 0 [104.86922885641631]
44 1 104.97058932871923 0 [104.86922885641631, 104.97058932871923]
25 0 105.49239695417266 0 [104.86922885641631, 104.97058932871923]
87 -1 105.84723980366932 0.9780109472530114 [104.97058932871923]
4 1 105.74398196773598 0 [104.97058932871923, 105.74398196773598]
75 -1 106.62993628427957 1.6593469555603377 [105.74398196773598]
82 0 106.94044691054229 0 [105.74398196773598]
21 -1 107.1365054926981 1.392523524

87 0 99.56392762270788 0 [101.01615399624097, 100.24162960022974, 99.7822482561871, 99.70796070042131, 99.51662823117351, 99.16155213786942, 99.4685804514114, 98.95536970755228, 99.52667978541253]
30 -1 100.39179019228614 -0.6243638039548358 [100.24162960022974, 99.7822482561871, 99.70796070042131, 99.51662823117351, 99.16155213786942, 99.4685804514114, 98.95536970755228, 99.52667978541253]
50 1 100.44640403190496 0 [100.24162960022974, 99.7822482561871, 99.70796070042131, 99.51662823117351, 99.16155213786942, 99.4685804514114, 98.95536970755228, 99.52667978541253, 100.44640403190496]
90 -1 100.68454250878438 0.44291290855463217 [99.7822482561871, 99.70796070042131, 99.51662823117351, 99.16155213786942, 99.4685804514114, 98.95536970755228, 99.52667978541253, 100.44640403190496]
29 1 100.5574090463195 0 [99.7822482561871, 99.70796070042131, 99.51662823117351, 99.16155213786942, 99.4685804514114, 98.95536970755228, 99.52667978541253, 100.44640403190496, 100.5574090463195]
75 -1 100.70750

59 0 95.20174868280627 0 [95.11151941607687, 95.41641783978062, 95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956]
9 -1 96.50681200208132 1.3952925860044445 [95.41641783978062, 95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956]
1 0 96.479995933088 0 [95.41641783978062, 95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956]
57 -1 96.46746165784151 1.05104381806089 [95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956]
28 0 96.28951986584794 0 [95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956]
50 0 96.52287988609686 0 [95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956]
81 1 97.14687930169077 0 [95.19509185394135, 95.55343134517774, 95.6055428248823, 93.89407092804703, 92.97155131164956, 97.14687930169077]
82 0 97.11955412543668 0 [95.195091

62 0 98.1024604029967 0 [101.03055017313417, 98.20877507669314]
94 -1 97.87864341438875 -3.151906758745426 [98.20877507669314]
32 0 97.44674821758176 0 [98.20877507669314]
74 1 97.41046586357565 0 [98.20877507669314, 97.41046586357565]
91 0 97.67351285317328 0 [98.20877507669314, 97.41046586357565]
17 0 97.56025639136594 0 [98.20877507669314, 97.41046586357565]
42 -1 97.00551701985499 -1.203258056838152 [97.41046586357565]
71 0 98.60817182314953 0 [97.41046586357565]
94 -1 98.91370439985192 1.503238536276271 []
23 0 98.42589736152318 0 []
49 -1 99.36172387719753 0 []
16 -1 98.47529763059035 0 []
21 0 97.84484333545002 0 []
87 1 97.59746487928022 0 [97.59746487928022]
72 -1 99.06015984932858 1.4626949700483607 []
88 -1 98.7489825187006 0 []
50 1 98.92671780691201 0 [98.92671780691201]
21 -1 98.64898160975089 -0.2777361971611185 []
60 1 98.80951916469482 0 [98.80951916469482]
78 1 100.28129689425127 0 [98.80951916469482, 100.28129689425127]
42 1 100.48823217514611 0 [98.80951916469482, 1

61 0 101.64369619429573 0 [99.06869058250835, 100.0361891334046]
31 -1 101.53734918816478 2.4686586056564295 [100.0361891334046]
78 -1 101.54815155088079 1.5119624174761839 []
42 1 102.75810612013194 0 [102.75810612013194]
65 1 102.3529863409598 0 [102.75810612013194, 102.3529863409598]
60 -1 102.07493098811393 -0.6831751320180075 [102.3529863409598]
38 -1 102.08170905571993 -0.2712772852398757 []
30 0 102.78269791245924 0 []
11 0 102.22574146774122 0 []
13 -1 102.17505352772216 0 []
74 -1 101.61378324589352 0 []
61 -1 100.08074088944012 0 []
37 -1 99.88458933313586 0 []
41 -1 99.58584810378166 0 []
53 0 99.48324695178412 0 []
85 1 98.74233020851983 0 [98.74233020851983]
96 -1 98.2521333667725 -0.49019684174733413 []
35 -1 98.02674543460854 0 []
45 1 97.89859140883271 0 [97.89859140883271]
56 -1 97.61285242974155 -0.28573897909116397 []
45 -1 98.26503756609334 0 []
28 1 98.15039333953253 0 [98.15039333953253]
9 -1 97.63324528638108 -0.5171480531514447 []
56 1 97.61920487718773 0 [97.61

...

85 0 101.6274879636503 0 [100.16118913789373]
33 1 101.1563705842641 0 [100.16118913789373, 101.1563705842641]
57 1 100.2801103568472 0 [100.16118913789373, 101.1563705842641, 100.2801103568472]
71 0 100.56755984651774 0 [100.16118913789373, 101.1563705842641, 100.2801103568472]
99 -1 101.45235194140201 1.2911628035082856 [101.1563705842641, 100.2801103568472]
30 0 101.62845703160947 0 [101.1563705842641, 100.2801103568472]
97 -1 101.14436684332536 -0.012003740938737906 [100.2801103568472]
84 -1 100.95102873105238 0.6709183742051721 []
27 1 101.18664470791404 0 [101.18664470791404]
14 0 101.1626546595078 0 [101.18664470791404]
53 0 102.13096934381959 0 [101.18664470791404]
5 0 103.55032397176419 0 [101.18664470791404]
31 -1 103.82042440665586 2.6337796987418187 []
99 0 103.55209609282996 0 []
78 0 103.24206867362359 0 []
36 0 104.49472668293448 0 []
16 1 104.75985574254884 0 [104.75985574254884]
94 1 104.04990751410091 0 [104.75985574254884, 104.04990751410091]
69 -1 105.2197716419 0.4

0 0 98.93344485934158 0 [99.14738373940673]
65 -1 98.71756147703825 -0.4298222623684893 []
25 1 97.96599676562786 0 [97.96599676562786]
49 -1 98.43447687725092 0.4684801116230517 []
5 -1 98.59002140344572 0 []
65 0 98.99152475222886 0 []
74 1 97.82848380664166 0 [97.82848380664166]
76 0 98.01834378268164 0 [97.82848380664166]
17 1 97.97350118749009 0 [97.82848380664166, 97.97350118749009]
32 0 98.37058137137194 0 [97.82848380664166, 97.97350118749009]
89 1 97.90826218875341 0 [97.82848380664166, 97.97350118749009, 97.90826218875341]
52 0 97.74788370644187 0 [97.82848380664166, 97.97350118749009, 97.90826218875341]
92 0 98.29956435225183 0 [97.82848380664166, 97.97350118749009, 97.90826218875341]
35 -1 99.10735878574918 1.2788749791075276 [97.97350118749009, 97.90826218875341]
62 0 99.30955675435908 0 [97.97350118749009, 97.90826218875341]
70 1 98.98446744985351 0 [97.97350118749009, 97.90826218875341, 98.98446744985351]
71 0 100.76773009873885 0 [97.97350118749009, 97.90826218875341, 9

77 -1 91.46884811726511 -0.5028752649099602 [92.22532099658463, 92.72695378857004, 92.71018825565483, 91.2542126981255, 91.20735721110817]
40 1 90.97458748054204 0 [92.22532099658463, 92.72695378857004, 92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
80 -1 91.89623472000847 -0.3290862765761631 [92.72695378857004, 92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
33 -1 90.92236375730027 -1.8045900312697682 [92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
72 0 90.38298283437646 0 [92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
89 0 90.5300182551836 0 [92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
77 0 90.11999786917056 0 [92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
79 0 89.75922833690892 0 [92.71018825565483, 91.2542126981255, 91.20735721110817, 90.97458748054204]
30 1 89.86800967608747 0 [92.71018825565483, 91.254212698125

62 -1 100.27088325917627 0 []
25 0 100.22403996194122 0 []
23 0 100.24524133696968 0 []
8 1 99.68240559485163 0 [99.68240559485163]
81 -1 100.83871542539814 1.1563098305465047 []
7 -1 100.9012511862857 0 []
58 0 99.65286349131182 0 []
48 0 99.51828192455818 0 []
25 1 99.91160281722568 0 [99.91160281722568]
24 -1 100.3153709191954 0.40376810196971746 []
93 -1 101.11130103538379 0 []
46 -1 100.83484238275983 0 []
13 -1 100.68740699783221 0 []
62 -1 100.838209047757 0 []
58 0 100.9114540298851 0 []
87 0 100.87566517561498 0 []
29 1 100.77760616140202 0 [100.77760616140202]
17 -1 100.33218752610983 -0.44541863529218517 []
18 -1 100.11144838594855 0 []
24 -1 100.1807571893533 0 []
45 -1 100.52815154699732 0 []
55 -1 99.47849590452424 0 []
13 1 99.37820562323012 0 [99.37820562323012]
48 1 99.65629371955153 0 [99.37820562323012, 99.65629371955153]
45 0 100.8243012686037 0 [99.37820562323012, 99.65629371955153]
35 1 100.06520758438852 0 [99.37820562323012, 99.65629371955153, 100.06520758438852

82 1 98.8126509441077 0 [99.5533666305743, 99.08482043005668, 99.22484149977238, 99.37716320412353, 98.8126509441077]
2 0 99.309974930396 0 [99.5533666305743, 99.08482043005668, 99.22484149977238, 99.37716320412353, 98.8126509441077]
75 -1 98.09986299253288 -1.4535036380414255 [99.08482043005668, 99.22484149977238, 99.37716320412353, 98.8126509441077]
52 0 98.58480860188493 0 [99.08482043005668, 99.22484149977238, 99.37716320412353, 98.8126509441077]
48 -1 98.26830728177251 -0.816513148284173 [99.22484149977238, 99.37716320412353, 98.8126509441077]
88 1 98.94419914583867 0 [99.22484149977238, 99.37716320412353, 98.8126509441077, 98.94419914583867]
96 1 97.88871657129744 0 [99.22484149977238, 99.37716320412353, 98.8126509441077, 98.94419914583867, 97.88871657129744]
58 -1 97.4226206707233 -1.8022208290490767 [99.37716320412353, 98.8126509441077, 98.94419914583867, 97.88871657129744]
18 0 97.78488009418156 0 [99.37716320412353, 98.8126509441077, 98.94419914583867, 97.88871657129744]
25 -

In [75]:
    
# This you need to write after training your MDP, this should just take in the state and return the Action you would perform
# Please do not make use of the returnNext function inside here, that would defeat the purpose of training the model, as it would be known to you!

def Run(state):
    s = state[0]*10+ state[1]%10
    action =getState(s, epsilon)
    return action
    

In [74]:
# This is the main function, you don't need to tamper with it!
def mainRun(iter = 1000):
    initstate = (random.randint(0,9),random.randint(0,9))
    i = 0
    initprice = 100.00
    price = initprice
    balance = 0
    bondbal = 0
    networth = 0
    st = initstate
    while i < iter:
        act = Run(st)
        pricCh, ns = returnNext(st,act)
        price += pricCh
        if act==1:
            balance -= 10*price
            bondbal += 10
        elif act==-1:
            balance += 10*price
            bondbal -= 10
        
        networth = balance + bondbal*price
        st = ns
        print('Your Networth has went from 0 to ',networth)
        i+=1

In [73]:

# ftr = input('Test : Filename to read from?')
# fte = input('Train : Filename to read?')

fte ="./test.pic"
ftr ="./train.pic"


with open(ftr,'rb') as f:
    Ztr = pickle.load(f)

with open(fte,'rb') as f:
    Zte = pickle.load(f)

with open('correlations.npy','rb') as f:
    corr = np.load(f)

In [84]:
ifTest =True
mainRun()

Your Networth has went from 0 to  0.0
Your Networth has went from 0 to  9.318639886374285
Your Networth has went from 0 to  9.318639886374285
Your Networth has went from 0 to  8.970706282339052
Your Networth has went from 0 to  16.27974980195404
Your Networth has went from 0 to  15.63165940823751
Your Networth has went from 0 to  15.63165940823751
Your Networth has went from 0 to  15.991210174570597
Your Networth has went from 0 to  17.492595996722685
Your Networth has went from 0 to  8.80951487754919
Your Networth has went from 0 to  34.83797135062969
Your Networth has went from 0 to  55.885614776892
Your Networth has went from 0 to  71.65601454587477
Your Networth has went from 0 to  45.566852056458174
Your Networth has went from 0 to  42.757212454499495
Your Networth has went from 0 to  31.74360595689086
Your Networth has went from 0 to  25.457812001171988
Your Networth has went from 0 to  25.717328030279305
Your Networth has went from 0 to  32.08957258924556
Your Networth has went 

Your Networth has went from 0 to  -838.497335957989
Your Networth has went from 0 to  -468.19385292018706
Your Networth has went from 0 to  -792.3435477612074
Your Networth has went from 0 to  -595.3873322752479
Your Networth has went from 0 to  -580.0181241959071
Your Networth has went from 0 to  -104.28770571034693
Your Networth has went from 0 to  -74.0398271746526
Your Networth has went from 0 to  -119.86047564190812
Your Networth has went from 0 to  -197.2317990588199
Your Networth has went from 0 to  -118.60071968733973
Your Networth has went from 0 to  348.8985222034098
Your Networth has went from 0 to  1272.7428190557694
Your Networth has went from 0 to  2053.970101260158
Your Networth has went from 0 to  2491.4647849313187
Your Networth has went from 0 to  2821.877279002787
Your Networth has went from 0 to  2967.3323030062165
Your Networth has went from 0 to  2917.5664740829816
Your Networth has went from 0 to  3225.29950459917
Your Networth has went from 0 to  3494.5186893085